This notebook is a helper to prepare the data for the dashboard construction

----

In [1]:
import duckdb
import pandas

In [2]:
# Load directly into DuckDB from CSV
con = duckdb.connect()
con.execute("""
    CREATE OR REPLACE TABLE term_deposit AS
    SELECT * FROM read_csv_auto('term-deposit-marketing.csv', header=True)
""")

In [3]:
# Filter the pre-call features for client segmentation and the subscribers (y=1)
con.execute("""
    CREATE OR REPLACE VIEW term_deposit_clean AS
    SELECT
        age,
        job,
        marital,
        education,
        "default" AS has_default,
        balance,
        housing AS has_housing,
        loan AS has_loan,
        y AS target
    FROM term_deposit
    WHERE target = True;
""")

In [4]:
con.execute("""
    SELECT
    COUNT(*)
    FROM term_deposit_clean;
""").fetchdf()

,count_star()
0,2896


In [5]:
# Check the first few rows
con.execute("SELECT * FROM term_deposit_clean LIMIT 10").fetchdf()


,age,job,marital,education,has_default,balance,has_housing,has_loan,target
0,59,admin,married,secondary,False,2343,True,False,True
1,56,admin,married,secondary,False,45,False,False,True
2,41,technician,married,secondary,False,1270,True,False,True
3,55,services,married,secondary,False,2476,True,False,True
4,54,admin,married,tertiary,False,184,False,False,True
5,42,management,single,tertiary,False,0,True,True,True
6,56,management,married,tertiary,False,830,True,True,True
7,60,retired,divorced,secondary,False,545,True,False,True
8,39,technician,single,unknown,False,45248,True,False,True
9,37,technician,married,secondary,False,1,True,False,True


In [6]:
mean_age, std_age, mean_balance, std_balance = con.execute("""
    SELECT
       AVG(age) AS mean_age,
       STDDEV_SAMP(age) AS std_age,
       AVG(LOG(ABS(balance) + 1)) AS mean_balance,
       STDDEV_SAMP(LOG(ABS(balance) + 1)) AS std_balance
    FROM term_deposit_clean
""").fetchone()


In [7]:
print(f"{mean_age=}\n{std_age=}\n{mean_balance=}\n{std_balance=}")

mean_age=39.84495856353591
std_age=11.001984220396205
mean_balance=2.630741327314328
std_balance=0.9629544120894596


In [8]:
con.execute(f"""
CREATE OR REPLACE VIEW term_deposit_clean_encoded AS
SELECT

    -- Pre-call original columns
    age,
    job,
    marital,
    education,
    has_default,
    balance,
    has_housing,
    has_loan,

    -- Standardize age manually
    (age - {mean_age}) / {std_age} AS age_t,

    -- Log-transform + standardize balance
    SIGN(balance) AS balance_sign,
    (LOG(ABS(balance) + 1) - {mean_balance}) / {std_balance} AS balance_t,

    -- One-hot encode job
    (job = 'admin.')::INT AS job_admin,
    (job = 'blue-collar')::INT AS job_bluecollar,
    (job = 'technician')::INT AS job_technician,
    (job = 'services')::INT AS job_services,
    (job = 'management')::INT AS job_management,
    (job = 'retired')::INT AS job_retired,
    (job = 'self-employed')::INT AS job_selfemployed,
    (job = 'entrepreneur')::INT AS job_entrepreneur,
    (job = 'unemployed')::INT AS job_unemployed,
    (job = 'housemaid')::INT AS job_housemaid,
    (job = 'student')::INT AS job_student,
    (job = 'unknown')::INT AS job_unknown,

    -- One-hot encode marital
    (marital = 'married')::INT AS marital_married,
    (marital = 'single')::INT AS marital_single,
    (marital = 'divorced')::INT AS marital_divorced,
    (marital = 'unknown')::INT AS marital_unknown,

    -- Ordinal encode education
    CASE 
        WHEN education = 'unknown' THEN -1
        WHEN education = 'primary' THEN 0
        WHEN education = 'secondary' THEN 1
        WHEN education = 'tertiary' THEN 2
    END AS education_ord,

    -- Binary encode default, housing, loan
    (has_default = 'yes')::INT AS default_bin,
    (has_housing = 'yes')::INT AS housing_bin,
    (has_loan = 'yes')::INT AS loan_bin,

    -- Binary encode target
    (target = 'yes')::INT AS target_bin

FROM term_deposit_clean
""")

In [9]:
con.execute("SELECT * FROM term_deposit_clean_encoded LIMIT 10").fetchdf()

,age,job,marital,education,has_default,balance,has_housing,has_loan,age_t,balance_sign,...,job_unknown,marital_married,marital_single,marital_divorced,marital_unknown,education_ord,default_bin,housing_bin,loan_bin,target_bin
0,59,admin,married,secondary,False,2343,True,False,1.741053,1,...,0,1,0,0,0,1,0,1,0,1
1,56,admin,married,secondary,False,45,False,False,1.468375,1,...,0,1,0,0,0,1,0,0,0,1
2,41,technician,married,secondary,False,1270,True,False,0.104985,1,...,0,1,0,0,0,1,0,1,0,1
3,55,services,married,secondary,False,2476,True,False,1.377483,1,...,0,1,0,0,0,1,0,1,0,1
4,54,admin,married,tertiary,False,184,False,False,1.286590,1,...,0,1,0,0,0,2,0,0,0,1
5,42,management,single,tertiary,False,0,True,True,0.195878,0,...,0,0,1,0,0,2,0,1,1,1
6,56,management,married,tertiary,False,830,True,True,1.468375,1,...,0,1,0,0,0,2,0,1,1,1
7,60,retired,divorced,secondary,False,545,True,False,1.831946,1,...,0,0,0,1,0,1,0,1,0,1
8,39,technician,single,unknown,False,45248,True,False,-0.076801,1,...,0,0,1,0,0,-1,0,1,0,1
9,37,technician,married,secondary,False,1,True,False,-0.258586,1,...,0,1,0,0,0,1,0,1,0,1


In [10]:
df_base = con.execute("SELECT * FROM term_deposit_clean_encoded").fetchdf()

In [11]:
df_before_kmeans = con.execute(f"""
    SELECT
        age_t,
        balance_sign,
        balance_t,
        job_admin,
        job_bluecollar,
        job_technician,
        job_services,
        job_management,
        job_retired,
        job_selfemployed,
        job_entrepreneur,
        job_unemployed,
        job_housemaid,
        job_student,
        job_unknown,
        marital_married,
        marital_single,
        marital_divorced,
        marital_unknown,
        education_ord,
        default_bin,
        housing_bin,
        loan_bin,
    FROM term_deposit_clean_encoded;
    """).df()

In [12]:
df_before_kmeans.shape

(2896, 23)

In [13]:
df_before_kmeans.to_csv("dashboard_df_before.csv", index=False)

In [14]:
from sklearn.cluster import KMeans
import numpy as np


In [15]:
seed = 23  # same seed used for the notebook part 2

In [16]:
kmeans_5 = KMeans(n_clusters=5, n_init='auto', random_state=seed).fit(df_before_kmeans)
labels_5 = kmeans_5.labels_


In [17]:
unique_values, counts = np.unique(labels_5, return_counts=True)

print("Values:", unique_values)
print("Counts:", counts)

Values: [0 1 2 3 4]
Counts: [802 553 790 450 301]


In [18]:
df_before_kmeans['cluster_k5'] = labels_5

In [19]:
df_before_kmeans.head()

,age_t,balance_sign,balance_t,job_admin,job_bluecollar,job_technician,job_services,job_management,job_retired,job_selfemployed,...,job_unknown,marital_married,marital_single,marital_divorced,marital_unknown,education_ord,default_bin,housing_bin,loan_bin,cluster_k5
0,1.741053,1,0.767654,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,0,1
1,1.468375,1,-1.005223,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
2,0.104985,1,0.491616,0,0,1,0,0,0,0,...,0,1,0,0,0,1,0,1,0,2
3,1.377483,1,0.792545,0,0,0,1,0,0,0,...,0,1,0,0,0,1,0,1,0,1
4,1.286590,1,-0.377556,0,0,0,0,0,0,0,...,0,1,0,0,0,2,0,0,0,1


In [20]:
# Adding the labels_5 clustering results as a new column to the base DataFrame
df_base['cluster_k5'] = labels_5

In [21]:
df_base.head()

,age,job,marital,education,has_default,balance,has_housing,has_loan,age_t,balance_sign,...,marital_married,marital_single,marital_divorced,marital_unknown,education_ord,default_bin,housing_bin,loan_bin,target_bin,cluster_k5
0,59,admin,married,secondary,False,2343,True,False,1.741053,1,...,1,0,0,0,1,0,1,0,1,1
1,56,admin,married,secondary,False,45,False,False,1.468375,1,...,1,0,0,0,1,0,0,0,1,1
2,41,technician,married,secondary,False,1270,True,False,0.104985,1,...,1,0,0,0,1,0,1,0,1,2
3,55,services,married,secondary,False,2476,True,False,1.377483,1,...,1,0,0,0,1,0,1,0,1,1
4,54,admin,married,tertiary,False,184,False,False,1.286590,1,...,1,0,0,0,2,0,0,0,1,1


In [22]:
df_base.to_csv("dashboard_base_k5.csv", index=False)

----

#### Prepare the table for the Radar visualization

In [23]:
grouped = df_base.groupby('cluster_k5').agg({
    'age': 'mean',
    'balance': 'mean',
    'education_ord': 'mean'
}).reset_index()
    


In [24]:
grouped.head()

,cluster_k5,age,balance,education_ord
0,0,31.139651,1795.230673,1.457606
1,1,56.439421,2322.171790,0.875226
2,2,37.536709,1322.343038,0.608861
3,3,39.553333,1850.311111,1.944444
4,4,39.046512,-3.132890,1.232558


In [25]:
# Rename columns for clarity
grouped.columns = ['cluster_k5', 'avg_age', 'avg_balance', 'avg_education_ord']

In [26]:
population_avg = pandas.DataFrame([{
    'cluster_k5': -1,
    'avg_age': df_base['age'].mean(),
    'avg_balance': df_base['balance'].mean(),
    'avg_education_ord': df_base['education_ord'].mean()
}])

In [27]:
combined = pandas.concat([grouped, population_avg], ignore_index=True)

In [28]:
combined[['avg_age', 'avg_balance', 'avg_education_ord']] = combined[['avg_age', 'avg_balance', 'avg_education_ord']].round(2)

In [29]:
from sklearn.preprocessing import MinMaxScaler


In [30]:
scaler = MinMaxScaler()
normalized_values = scaler.fit_transform(combined[['avg_age', 'avg_balance', 'avg_education_ord']])
normalized_df = pandas.DataFrame(normalized_values, columns=['norm_age', 'norm_balance', 'norm_education_ord'])




In [31]:
combined_final = pandas.concat([combined, normalized_df], axis=1)


In [32]:
combined_final

,cluster_k5,avg_age,avg_balance,avg_education_ord,norm_age,norm_balance,norm_education_ord
0,0,31.14,1795.23,1.46,0.000000,0.773388,0.639098
1,1,56.44,2322.17,0.88,1.000000,1.000000,0.203008
2,2,37.54,1322.34,0.61,0.252964,0.570021,0.000000
3,3,39.55,1850.31,1.94,0.332411,0.797076,1.000000
4,4,39.05,-3.13,1.23,0.312648,0.000000,0.466165
5,-1,39.84,1588.50,1.17,0.343874,0.684484,0.421053


In [33]:
# Step 7: Create long format with norm and abs values
df_abs = pandas.melt(combined_final, id_vars='cluster_k5',
                 value_vars=['avg_age', 'avg_balance', 'avg_education_ord'],
                 var_name='metric_type', value_name='abs_value')

df_norm = pandas.melt(combined_final, id_vars='cluster_k5',
                  value_vars=['norm_age', 'norm_balance', 'norm_education_ord'],
                  var_name='metric_type', value_name='norm_value')



In [34]:
df_abs

,cluster_k5,metric_type,abs_value
0,0,avg_age,31.14
1,1,avg_age,56.44
2,2,avg_age,37.54
3,3,avg_age,39.55
4,4,avg_age,39.05
5,-1,avg_age,39.84
6,0,avg_balance,1795.23
7,1,avg_balance,2322.17
8,2,avg_balance,1322.34
9,3,avg_balance,1850.31


In [35]:
df_norm

,cluster_k5,metric_type,norm_value
0,0,norm_age,0.000000
1,1,norm_age,1.000000
2,2,norm_age,0.252964
3,3,norm_age,0.332411
4,4,norm_age,0.312648
5,-1,norm_age,0.343874
6,0,norm_balance,0.773388
7,1,norm_balance,1.000000
8,2,norm_balance,0.570021
9,3,norm_balance,0.797076


In [36]:
# Clean metric_type in normalized version
df_norm['metric_type'] = df_norm['metric_type'].str.replace('norm_', 'avg_')

# Step 8: Merge both on cluster + metric
df_radar = pandas.merge(df_norm, df_abs, on=['cluster_k5', 'metric_type'])


In [37]:
df_radar

,cluster_k5,metric_type,norm_value,abs_value
0,0,avg_age,0.000000,31.14
1,1,avg_age,1.000000,56.44
2,2,avg_age,0.252964,37.54
3,3,avg_age,0.332411,39.55
4,4,avg_age,0.312648,39.05
5,-1,avg_age,0.343874,39.84
6,0,avg_balance,0.773388,1795.23
7,1,avg_balance,1.000000,2322.17
8,2,avg_balance,0.570021,1322.34
9,3,avg_balance,0.797076,1850.31


In [38]:
df_radar.to_csv("radar_linearchart.csv", index=False)